<a href="https://colab.research.google.com/github/tauhid017/GreenSkills/blob/main/Wildfire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -upgrade pip


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [2]:
# Install opencv (using the correct package name)
!pip install opencv-python

# Import necessary modules (assuming open_code is a custom function)
try:
  from open_code import open_code
except ImportError:
  print("The 'open_code' function is not found. Make sure it's installed or defined.")

# You can now use opencv and open_code (if available) in your code

The 'open_code' function is not found. Make sure it's installed or defined.


In [3]:

pip install opencv-contrib-python

In [4]:

import cv2
print(cv2.__version__)

4.11.0


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wildfire-prediction-dataset


In [14]:
train_dir = r"/kaggle/input/wildfire-prediction-dataset/train"
valid_dir = r"/kaggle/input/wildfire-prediction-dataset/valid"
test_dir =r"/kaggle/input/wildfire-prediction-dataset/test"

#set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255, fill_mode='nearest')

#load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [20]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
  history = model.fit(train_generator, epochs=10, validation_data=valid_generator, verbose=1)

Epoch 1/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.8910 - loss: 0.2709

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 262s 275ms/step - accuracy: 0.8910 - loss: 0.2709 - val_accuracy: 0.9467 - val_loss: 0.1485
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 237s 251ms/step - accuracy: 0.9329 - loss: 0.1822 - val_accuracy: 0.9363 - val_loss: 0.1686
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 236s 249ms/step - accuracy: 0.9430 - loss: 0.1561 - val_accuracy: 0.9562 - val_loss: 0.1188
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 258s 272ms/step - accuracy: 0.9455 - loss: 0.1423 - val_accuracy: 0.9575 - val_loss: 0.1142
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 238s 251ms/step - accuracy: 0.9568 - loss: 0.1159 - val_accuracy: 0.9519 - val_loss: 0.1269
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 254s 268ms/step - accuracy: 0.9575 - loss: 0.1155 - val_accuracy: 0.9573 - val_loss: 0.1182
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 253s 268ms/step - accuracy: 0.9662 - loss: 0.0901 - val_accuracy: 0.9567 - val_loss: 0.1279
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 253s 268ms/step - accuracy: 0.9719 - loss: 0.08

In [23]:
model.save('fire_model.h5')

In [24]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)
        image_label.configure(image=img)
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0]
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image)
btn.pack(pady=20)

image_label = tk.Label(root)
image_label.pack()

result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [26]:
!pip install gradio

In [27]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('fire_model.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated input definition
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains a wildfire."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98082d156973e4a5f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**To use this script:**

1.  Save the code above as a Python file (e.g., `predict_wildfire.py`) in the same directory where you saved your `fire_model.h5` file.
2.  Open a terminal or command prompt.
3.  Navigate to the directory where you saved the file.
4.  Run the script using the following command, replacing `path/to/your/image.jpg` with the actual path to the image you want to predict on: